# Graph Powered Machine Learning Exercise 1

### Group Members:
1. Kashif Rabbani
2. Emil Riss Hensin
3. Jonas Brusokas


Link to the exercise pdf file:  https://github.com/joerg84/Graph_Powered_ML_Workshop

In [24]:
!pip3 install rdflib
!pip3 install numpy

In [25]:
# Import statements
import rdflib
from rdflib import Graph, Namespace
from rdflib.namespace import DC, RDF, FOAF, RDFS, RDF, XSD
from rdflib import URIRef, BNode, Literal
import numpy as np

**1. Create an RDF Graph representing the same road network and travel
times**

In [26]:
g = Graph()
ex = Namespace("http://example.org/travel/")
g.bind("ex", ex)

g.add((URIRef(ex.City), RDF.type, RDFS.Class))
g.add((URIRef(ex.Connection), RDF.type, RDFS.Class))

g.add((ex.fromCity, RDF.type, RDF.Property))
g.add((ex.toCity, RDF.type, RDF.Property))
g.add((ex.travelTime, RDF.type, RDF.Property))
g.add((ex.hasConnection, RDF.type, RDF.Property))

g.add((ex.fromCity, RDFS.domain, ex.Connection))
g.add((ex.toCity, RDFS.range, ex.City))

g.add((ex.toCity, RDFS.domain, ex.Connection))
g.add((ex.fromCity, RDFS.range, ex.City))

g.add((ex.travelTime, RDFS.domain, ex.Connection))
g.add((ex.travelTime, RDFS.range, XSD.decimal))

g.add((ex.hasConnection, RDFS.domain, ex.City))
g.add((ex.hasConnection, RDFS.range, ex.City))

paths = [("Inverness", "Aberdeen", 3, 2.5),
         ("Aberdeen", "Leuchars", 1.5, 1),
         ("Leuchars", "StAndrews", 0.2, 0.2),
         ("Leuchars", "Edinburgh", 1.5, 3),
         ("Glasgow", "Edinburgh", 1, 1),
         ("Glasgow", "Carlisle", 1, 1),
         ("York", "Carlisle", 3.5, 2.5),
         ("Birmingham", "Carlisle", 1, 2),
         ("Birmingham", "London", 1.5, 2.5),
         ("York", "Edinburgh", 4, 3.5),
         ("York", "London", 1.8, 2),
         ("Brussels", "London", 3.5, 2.5),
         ("Brussels", "Cologne", 2, 1.5),
         ("Toronto", "Winnipeg", 36, 35),
         ("Winnipeg", "Saskatoon", 12, 5),
         ("Saskatoon", "Edmonton", 12, 17),
         ("Edmonton", "Jasper", 6, 5),
         ("Jasper", "Vancouver", 12, 13)]


city_list1 = list(set([city[0] for city in paths]))
city_list2 = list(set([city[1] for city in paths]))
all_cities = set(city_list1 + city_list2)

for city in all_cities:
    g.add((URIRef(ex[city]), RDF.type, ex.City))
    g.add((URIRef(ex[city]), RDFS.label, Literal(f"{city}^^xsd:string")))

i = 0
for path in paths:
    g.add((URIRef(ex[f"con{i}"]), RDF.type, URIRef(ex["Connection"])))
    g.add((URIRef(ex[f"con{i}"]), ex.fromCity, URIRef(ex[f"{path[0]}"])))
    g.add((URIRef(ex[f"con{i}"]), ex.toCity, URIRef(ex[f"{path[1]}"])))
    g.add((URIRef(ex[f"con{i}"]), ex.travelTime, Literal(f"{path[2]}", datatype=XSD.decimal)))
    g.add((URIRef(ex[f"{path[0]}"]), ex.hasConnection, (URIRef(ex[f"{path[1]}"]))))
    i += 1

    g.add((URIRef(ex[f"con{i}"]), RDF.type, URIRef(ex["Connection"])))
    g.add((URIRef(ex[f"con{i}"]), ex.fromCity, URIRef(ex[f"{path[1]}"])))
    g.add((URIRef(ex[f"con{i}"]), ex.toCity, URIRef(ex[f"{path[0]}"])))
    g.add((URIRef(ex[f"con{i}"]), ex.travelTime, Literal(f"{path[3]}", datatype=XSD.decimal)))
    g.add((URIRef(ex[f"{path[1]}"]), ex.hasConnection, URIRef(ex[f"{path[0]}"])))
    i += 1

# Print the graph in turtle format
print(g.serialize(format="turtle").decode("utf-8"))

@prefix ex: <http://example.org/travel/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

ex:City a rdfs:Class .

ex:Connection a rdfs:Class .

ex:con0 a ex:Connection ;
    ex:fromCity ex:Inverness ;
    ex:toCity ex:Aberdeen ;
    ex:travelTime 3.0 .

ex:con1 a ex:Connection ;
    ex:fromCity ex:Aberdeen ;
    ex:toCity ex:Inverness ;
    ex:travelTime 2.5 .

ex:con10 a ex:Connection ;
    ex:fromCity ex:Glasgow ;
    ex:toCity ex:Carlisle ;
    ex:travelTime 1.0 .

ex:con11 a ex:Connection ;
    ex:fromCity ex:Carlisle ;
    ex:toCity ex:Glasgow ;
    ex:travelTime 1.0 .

ex:con12 a ex:Connection ;
    ex:fromCity ex:York ;
    ex:toCity ex:Carlisle ;
    ex:travelTime 3.5 .

ex:con13 a ex:Connection ;
    ex:fromCity ex:Carlisle ;
    ex:toCity ex:York ;
    ex:travelTime 2.5 .

ex:con14 a ex:Connection ;
    ex:fromCity ex:Birmingham ;
    ex:toCity ex:Carlisle

**2. Implement a SPARQL query returning all cities which can be reached
from London.**

In [27]:
# Get all the Cities which can be reached from London
cities = g.query(
    """ SELECT DISTINCT ?city WHERE {
    ex:London ex:hasConnection+/ex:hasConnection ?city.  
    FILTER(?city != ex:London)
}
""", initNs={'ex': "http://example.org/travel/"})

for row in cities:
    print(row["city"])

http://example.org/travel/Cologne
http://example.org/travel/Brussels
http://example.org/travel/York
http://example.org/travel/Birmingham
http://example.org/travel/Carlisle
http://example.org/travel/Edinburgh
http://example.org/travel/Glasgow
http://example.org/travel/Leuchars
http://example.org/travel/Aberdeen
http://example.org/travel/StAndrews
http://example.org/travel/Inverness


**Bonus: all cities which can be reached from London within less than
5 hours.**

In [28]:
starting_city = URIRef(ex["London"])
global_dict = {}

def get_neighbours(city, time):
    query = """SELECT ?t ?c ?con WHERE {
                  ?con a ex:Connection ; 
                    ex:fromCity CITY  ; 
                    ex:travelTime ?t ;
                    ex:toCity ?c .
                  } """.replace("CITY", "ex:" + city)
    output = g.query(query, initNs={'ex': "http://example.org/travel/"})
    dict = {}
    for val in output:
        if val["c"] != city:
            dict[val["c"]] = float(val["t"]) + float(time)

    for x, y in dict.items():
        if y < 5:
            global_dict[x] = y
            get_neighbours(x.rsplit('/', 1)[1], y)


get_neighbours(starting_city.rsplit('/', 1)[1], 0)
for x, y in global_dict.items():
    if x != starting_city:
        print(x, y)

http://example.org/travel/York 2.0
http://example.org/travel/Birmingham 2.5
http://example.org/travel/Carlisle 3.5
http://example.org/travel/Glasgow 4.5
http://example.org/travel/Brussels 2.5
http://example.org/travel/Cologne 4.5


**3. Implement generic python code (i.e., the algorithms don't have to be
specied in SPARQL, but could be) for the Single Source Shortest Path
algorithm and return the shortest paths to all other cities starting from
London. You can choose either Dijkstra's or Bellman-Ford's algorithm.**



```
# At first we define two classes ,i.e., Node and Graph
```
Ref: [Article](https://medium.com/cantors-paradise/dijkstras-shortest-path-algorithm-in-python-d955744c7064)


In [29]:
class Node:

    def __init__(self, data, indexloc=None):
        self.data = data
        self.index = indexloc

class Graph:

    @classmethod
    def create_from_nodes(self, nodes):
        return Graph(len(nodes), len(nodes), nodes)

    def __init__(self, row, col, nodes=None):
        # set up an adjacency matrix
        self.adj_mat = [[0] * col for _ in range(row)]
        self.nodes = nodes
        for i in range(len(self.nodes)):
            self.nodes[i].index = i

    # Conncects from node1 to node2
    # Note row is source, column is destination
    # Updated to allow weighted edges (supporting dijkstra's alg)
    def connect_dir(self, node1, node2, weight=1):
        node1, node2 = self.get_index_from_node(node1), self.get_index_from_node(node2)
        self.adj_mat[node1][node2] = weight

    # Optional weight argument to support dijkstra's alg
    def connect(self, node1, node2, weight=1):
        self.connect_dir(node1, node2, weight)
        self.connect_dir(node2, node1, weight)

    # Get node row, map non-zero items to their node in the self.nodes array
    # Select any non-zero elements, leaving you with an array of nodes
    # which are connections_to (for a directed graph)
    # Return value: array of tuples (node, weight)
    def connections_from(self, node):
        node = self.get_index_from_node(node)
        return [(self.nodes[col_num], self.adj_mat[node][col_num]) for col_num in range(len(self.adj_mat[node])) if
                self.adj_mat[node][col_num] != 0]

    # Map matrix to column of node
    # Map any non-zero elements to the node at that row index
    # Select only non-zero elements
    # Note for a non-directed graph, you can use connections_to OR
    # connections_from
    # Return value: array of tuples (node, weight)
    def connections_to(self, node):
        node = self.get_index_from_node(node)
        column = [row[node] for row in self.adj_mat]
        return [(self.nodes[row_num], column[row_num]) for row_num in range(len(column)) if column[row_num] != 0]

    def print_adj_mat(self):
        for row in self.adj_mat:
            print(row)

    def node(self, index):
        return self.nodes[index]

    def remove_conn(self, node1, node2):
        self.remove_conn_dir(node1, node2)
        self.remove_conn_dir(node2, node1)

    # Remove connection in a directed manner (nod1 to node2)
    # Can accept index number OR node object
    def remove_conn_dir(self, node1, node2):
        node1, node2 = self.get_index_from_node(node1), self.get_index_from_node(node2)
        self.adj_mat[node1][node2] = 0

        # Can go from node 1 to node 2?

    def can_traverse_dir(self, node1, node2):
        node1, node2 = self.get_index_from_node(node1), self.get_index_from_node(node2)
        return self.adj_mat[node1][node2] != 0

    def has_conn(self, node1, node2):
        return self.can_traverse_dir(node1, node2) or self.can_traverse_dir(node2, node1)

    def add_node(self, node):
        self.nodes.append(node)
        node.index = len(self.nodes) - 1
        for row in self.adj_mat:
            row.append(0)
        self.adj_mat.append([0] * (len(self.adj_mat) + 1))

    # Get the weight associated with travelling from n1
    # to n2. Can accept index numbers OR node objects
    def get_weight(self, n1, n2):
        node1, node2 = self.get_index_from_node(n1), self.get_index_from_node(n2)
        return self.adj_mat[node1][node2]

    # Allows either node OR node indices to be passed into
    def get_index_from_node(self, node):
        if not isinstance(node, Node) and not isinstance(node, int):
            raise ValueError("node must be an integer or a Node object")
        if isinstance(node, int):
            return node
        else:
            return node.index

    def dijkstra(self, node):
        # Get index of node (or maintain int passed in)
        nodenum = self.get_index_from_node(node)
        # Make an array keeping track of distance from node to any node
        # in self.nodes. Initialize to infinity for all nodes but the
        # starting node, keep track of "path" which relates to distance.
        # Index 0 = distance, index 1 = node hops
        dist = [None] * len(self.nodes)
        for i in range(len(dist)):
            dist[i] = [float("inf")]
            dist[i].append([self.nodes[nodenum]])

        dist[nodenum][0] = 0
        # Queue of all nodes in the graph
        # Note the integers in the queue correspond to indices of node
        # locations in the self.nodes array
        queue = [i for i in range(len(self.nodes))]
        # Set of numbers seen so far
        seen = set()
        while len(queue) > 0:
            # Get node in queue that has not yet been seen
            # that has smallest distance to starting node
            min_dist = float("inf")
            min_node = None
            for n in queue:
                if dist[n][0] < min_dist and n not in seen:
                    min_dist = dist[n][0]
                    min_node = n

            # Add min distance node to seen, remove from queue
            queue.remove(min_node)
            seen.add(min_node)
            # Get all next hops
            connections = self.connections_from(min_node)
            # For each connection, update its path and total distance from
            # starting node if the total distance is less than the current distance
            # in dist array
            for (node, weight) in connections:
                tot_dist = weight + min_dist
                if tot_dist < dist[node.index][0]:
                    dist[node.index][0] = tot_dist
                    dist[node.index][1] = list(dist[min_node][1])
                    dist[node.index][1].append(node)
        return dist



```
# Querying the RDF Graph and getting the relevant cities and their info to run the shortest path algorithm
```



In [30]:
graphNodes = []
adjacent_cities = []
# nodes_reachable_from_london
queryA = """SELECT DISTINCT ?city WHERE {
            ex:London ex:hasConnection+/ex:hasConnection ?city.  
        }
        """
# adjacent_nodes_with_time
queryB = """SELECT distinct ?fromCity ?toCity ?time WHERE {
            ex:London ex:hasConnection+/ex:hasConnection ?toCity.  
            ?conA a ex:Connection ; 
                  ex:fromCity ?fromCity; 
                  ex:travelTime ?time ;
                  ex:toCity ?toCity .
        }"""

result_queryA = g.query(queryA, initNs={'ex': "http://example.org/travel/"})

result_queryB = g.query(queryB, initNs={'ex': "http://example.org/travel/"})

# Create Graph Nodes
for node in result_queryA:
    graphNodes.append(Node(node["city"].rsplit('/', 1)[1]))

# Create Graph of Cities
cities_graph = Graph.create_from_nodes(graphNodes)

# Get Adjacent Cities starting from London with their travel time
for row in result_queryB:
    tuple_row = (row["fromCity"].rsplit('/', 1)[1], row["toCity"].rsplit('/', 1)[1], Literal(row["time"]).value)
    adjacent_cities.append(tuple_row)

# Connect the Nodes with edges having weight (travel time) to create an adjacency Matrix
for edge in adjacent_cities:
    a = None
    b = None
    for v in graphNodes:
        if v.data == edge[0]:
            a = v

        if v.data == edge[1]:
            b = v

    cities_graph.connect(a.index, b.index, float(edge[2]))

# Print the adjacency Matrix
cities_graph.print_adj_mat()

# Run the dijkstra Algorithm starting from the node 0 i.e., London
result = [(weight, [n.data for n in node]) for (weight, node) in cities_graph.dijkstra(graphNodes[0])]

# See the shortest Paths
for r in result:
    print(r)

[0, 0, 2.5, 2.0, 1.5, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 1.5, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[2.5, 1.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[2.0, 0, 0, 0, 0, 2.5, 3.5, 0, 0, 0, 0, 0]
[1.5, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 2.5, 1.0, 0, 0, 1.0, 0, 0, 0, 0]
[0, 0, 0, 3.5, 0, 0, 0, 1.0, 3.0, 0, 0, 0]
[0, 0, 0, 0, 0, 1.0, 1.0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 3.0, 0, 0, 1.5, 0.2, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 1.5, 0, 0, 3.0]
[0, 0, 0, 0, 0, 0, 0, 0, 0.2, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 3.0, 0, 0]
(0, ['London'])
(4.0, ['London', 'Brussels', 'Cologne'])
(2.5, ['London', 'Brussels'])
(2.0, ['London', 'York'])
(1.5, ['London', 'Birmingham'])
(2.5, ['London', 'Birmingham', 'Carlisle'])
(4.5, ['London', 'Birmingham', 'Carlisle', 'Glasgow', 'Edinburgh'])
(3.5, ['London', 'Birmingham', 'Carlisle', 'Glasgow'])
(7.5, ['London', 'Birmingham', 'Carlisle', 'Glasgow', 'Edinburgh', 'Leuchars'])
(9.0, ['London', 'Birmingham', 'Carlisle', 'Glasgow', 'Edinburgh', 'Leuchars', 'Aberdeen'])
(7.7, ['London', 'Bi